# Data Wrangling (WeRateDogs Data)

![Image via Boston Magazine](https://d17h27t6h515a5.cloudfront.net/topher/2017/October/59dd378f_dog-rates-social/dog-rates-social.jpg)
<div style="text-align: center"><i>Image via <a href="https://www.bostonmagazine.com/arts-entertainment/2017/04/18/dog-rates-mit/">Boston Magazine</i></div>

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#gathering">Gathering</a></li>
<li><a href="#assessing">Assessing</a></li>
<li><a href="#cleaning">Cleaning</a></li>
<li><a href="#iterating">Iterating</a></li>
<li><a href="#storing">Storing</a></li>
<li><a href="#acting">Acting</a></li>
</ul>

<a id='intro'></a>
## Introduction
<div style="text-align:justify">
<br>
    <a href="https://twitter.com/dog_rates?lang=en">WeRateDogs</a> is a Twitter account that rates people's dogs with a humorous comment about the dog. Their Twitter account data as many other real world cases does not comes clean, so as part of the Udacity's Data Analyst Nanodegree program the data wrangling process must be applied in order to obtain data that will be useful to create interesting and trustworthy analyses and visualizations of these rates.
</div>

### Objectives

- Gathering: Gather data from three different sources in order to create a master dataset **(WeRateDogs Twitter archive, Image predictions file and Twitter API)**.

- Assesing: Assess both visually and programatically for quality and tidiness issues. Detect and document at least **8 quality issues and 2 tidiness issues**.

- Cleaning: Clean each of the issues you documented while assessing. **The result should be a high quality and tidy master pandas DataFrame (or DataFrames, if appropriate)**.

- Storing: Store the clean DataFrame(s) in a CSV file with the main one. **Additionally, store the cleaned data in a SQLite database**.

- Acting: Analyze and visualize the wrangled data, **producing at least 3 insights and 1 visualization**.

In [330]:
# prerequisite package imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

# library for HTTP request
import requests

# libaries for os interfaces interaction
import os
import filecmp

# library for Twitter API
#import tweepy
#from tweepy import OAuthHandler

# library to handle json
import json
import collections

# library for sql database
import sqlite3

# package to calculate time
from timeit import default_timer as timer

%matplotlib inline

<a id='gathering'></a>
## Gathering the Data

The first source that will be gather is the WeRateDogs Twitter archive, this file has been downloaded manually from the following url: [Data source 1](https://d17h27t6h515a5.cloudfront.net/topher/2017/August/59a4e958_twitter-archive-enhanced/twitter-archive-enhanced.csv).

After the download, I transferred the file to the * src * folder located in the main directory of this notebook. Once there we read the file using the `read_csv` function from Pandas.

In [ ]:
folder_name = 'src'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

twitter_archive = pd.read_csv('src/twitter-archive-enhanced.csv')
twitter_archive.head();

In [ ]:
#Then we look that the data has been loaded properly by checking its info
twitter_archive.info()

Now the second source that will be gather is the Image Predictions file, which is hosted on [Udacity's servers](https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv) and should be programmatically downloaded, for this purpose we use the `request` library.

In [ ]:
url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
#We here try to made a http GET request to obtain the files from the server
response = requests.get(url)
if response.status_code == 200:
    print('Sucessful request')
else:
    print('The request was not sucessful:' + str(response.status_code))

In [ ]:
# If the request was sucessful, now we proceed to save the tsv file
with open(os.path.join(folder_name,
                       url.split('/')[-1]), mode='wb') as file:
    file.write(response.content)

In [ ]:
# If the file has been sucessfully saved into the src folder, then we just
# have to assert that the number of files in that directory
assert len(os.listdir('src')) == 3

In [ ]:
# Finally we just have to read the file, using pandas
image_predictions = pd.read_csv('src/image-predictions.tsv', sep="\t")
image_predictions.head()

In [ ]:
#Then we look that the data has been loaded properly by checking its info
image_predictions.info()

Finally the third source will be gather using Twitter's API, the library `tweepy` allow us to to interact with the API using Python.

In [ ]:
# Tweet IDs for which to gather additional data via Twitter's API
tweet_ids = twitter_archive.tweet_id.values
len(tweet_ids)

In [ ]:
# The tokens for authetication must be HIDDEN for security reasons and
# comply with Twitter's Terms and Conditions
consumer_key = 'HIDDEN'
consumer_secret = 'HIDDEN'
access_token = 'HIDDEN'
access_token_secret = 'HIDDEN'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Then we create an instance of the API
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [ ]:
# Next we extract the ids from twitter-archive-enhanced dataset
query_ids = twitter_archive['tweet_id'].values
tweets = len(query_ids)

In [ ]:
# Once having all the ids, we query Twitter's API for information from each Tweet
# Twitter will return JSON data, so we must dump it into a file

# These variables will help to keep on check on how is going the process
# and retrieve failed tweets
gauge = 0
start = timer()
failed_tweets = {}

with open('src/tweets_json.txt', mode='w') as output:   
    for tweet_id in query_ids:
        gauge += 1
        # Progress message
        print("Retrieving: " + str(tweet_id) + 
              "  Missing: " + str(tweets-gauge))
        try:
            # Get the tweet query status
            # Pass in 'extended' to get non truncated tweet text
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            print("Sucess")
            json.dump(tweet.json, output)
            output.write("\n")
        # In case that Twitter responds with an error
        except tweepy.TweepError as e:
            print("Failure")
            # We make a dictionary with the tweets that throwed errors
            # for future handling
            failed_tweets[tweet_id] = e
            pass
end = timer()
print("Total Time: " + str(end-start))
print("Errors:")
print(failed_tweets)

In [ ]:
# Now we read the dumped txt file, and append each tweet in a list
tweets = []
for line in open('src/tweets_json.txt', 'r'):
    tweets.append(json.loads(line))
# We verify the amount of tweets
len(tweets)

In [ ]:
# Then we explore the structure of the tweet in order to find the information that we want:
tweets[0]

> Once the tweets have been appended to the list, we extract the additional data that we want so we can build a dictionary, and finally construct the dataframe. The variables that will be extracted are:
- *id*
- *retweet_count*
- *favorite_count*

In [ ]:
# We create lists that will hold each variable of the tweets
tweet_ids = []
retweet_counts = []
favorite_counts = []

for tweet in tweets:
    tweet_ids.append(tweet['id'])
    retweet_counts.append(tweet['retweet_count'])
    favorite_counts.append(tweet['favorite_count'])
    
# Now we construct the dataframe from a dictionary with the previous data
data = {'tweet_id':tweet_ids, 'retweet_count':retweet_counts, 'favorite_count':favorite_counts} 
tweets_counts = pd.DataFrame(data)
tweets_counts.info()

> Now we have all the sources and data required for this project, we proceed to assess each dataset in order to find quality and tideness issues.

<a id='assessing'></a>
## Assesing Data

### Visual Assessment
The very first step when assessing the data is to do it visually, even though this might not be effective for large datasets, it will help us to get acquainted with the data.

#### Twitter Archive Dataset

In [ ]:
# Display the twitter archive table
twitter_archive.head()

> Indentified issues:
- Missing values for columns: (retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp, in_reply_to_status_id, in_reply_to_user_id).
- There is no information about the type of dog, 'None' value for columns: (doggo, floofer, pupper, puppo).
- `source` column has embedded HTML code.

`twitter_archive` columns:
- **tweet_id**: is the tweet identifier.
- **in_reply_to_status_id**: reply status identifier.
- **in_reply_to_user_id**: reply to user identifier.
- **timestamp** the timestamp assigned to each tweet when it is created.
- **source**: the source (mobile) from which the tweet was made.
- **text**: the text (body) that each tweet has (during that time Twitter only allowed texts of 140 characters max).
- **retweeted_status_id**: identifier of the status if this tweet was retweeted.
- **retweeted_status_user_id**: identifier of the user if this tweet was retweeted.
- **retweeted_status_timestamp**: the timestamp given if this tweet was retweeted.
- **expanded_urls**: expanded urls.
- **rating_numerator**: numerator of the ranking, almost always above 10.
- **rating_denominator**: denominator of the ranking, fixed to 10.
- **name**: the given name to the dog.
- **doggo**: a type of dog. A big pupper, usually older. This label does not stop a doggo from behaving like a pupper.
- **floofer**: a type of dog. A any dog really. However, this label is commonly given to dogs with semmingly excess fur.
- **pupper**: a type of dog. A small doggo, usually younger. Can be equally, if not more mature than some doggos.
- **puppo**: a type of dog. A transitional state between pupper and doggo. Easily understood as the dog equivalent of a teenager.

**Definitions for dogs were taken from:** *TheDogtionary (via the #WeRateDogs book on Amazon)*

In [ ]:
# Display the image predictions table
image_predictions.head()

> Indentified issues:
- Some breeds are capitalized while others not, also instead of a space the character '_' is used, for columns: (p1, p2, p3)

`image_predictions` columns:
- **tweet_id**: is the tweet identifier.
- **jpg_url**: the url to the image of the dog used for the prediction.
- **img_num**: number of image.
- **p1**: p1 is the algorithm's #1 prediction for the image in the tweet.
- **p1_conf**: p1_conf is how confident the algorithm is in its #1 prediction.
- **p1_dog**: p1_dog is whether or not the #1 prediction is a breed of dog.
- **p2**: p2 is the algorithm's second most likely prediction.
- **p2_conf**: p2_conf is how confident the algorithm is in its #2 prediction.
- **p2_dog**: p2_dog is whether or not the #2 prediction is a breed of dog.
- **p3**: p3 is the algorithm's third most likely prediction.
- **p3_conf**: p2_conf is how confident the algorithm is in its #3 prediction.
- **p3_dog**: p2_dog is whether or not the #3 prediction is a breed of dog.

In [ ]:
# Display the tweets counts table
tweets_counts.head()

`tweets_counts` columns:
- **tweet_id**: is the tweet identifier.
- **retweet_count**: number of retweets of that tweet.
- **favorite_count**: number of favourites of that tweet.

### Programmatic Assesment
Now programmatically we will look for quality and tidiness issues in the datasets, using pandas functions.

### Twitter Archive

In [ ]:
# Look for dimensions
twitter_archive.info()

> Indentified issues:
- Remove retweets
- These columns are not needed since they are retweets: (retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp, in_reply_to_status_id, in_reply_to_user_id).
- Timestamp columns as strings, not datetime.

In [ ]:
# Now all the denominators are fixed to 10
twitter_archive.rating_denominator.value_counts()

**The previous information is pretty interesting, leets look further into these values, since is our main metric**

In [ ]:
# Lets look at those tweets with different denominators
twitter_archive[twitter_archive.rating_denominator < 10][['tweet_id', 'text', 'rating_numerator', 'rating_denominator']]

In [ ]:
# Something seems odd about text and the numerators, leets look at them
texts = twitter_archive[twitter_archive.rating_denominator < 10]['text'].values
for i in range(len(texts)):
    print("Text " + str(i+1)+ ":" + texts[i])

> From these tweets we can detect what was the correct rating. Except for the case of Sam, which actually refers that she smiles all the day.
- Tweets (835246439529840640, 666287406224695296) [13/10, 9/10] - Incorrect rating
- Tweet (810984652412424192) - Invalid rating

In [ ]:
twitter_archive[twitter_archive.rating_denominator > 100][['tweet_id', 'text', 'rating_numerator', 'rating_denominator']]

In [ ]:
# Both numerator and denominator seem to be pretty high, lets look at the text to see why.
# Something seems odd about text and the numerators, leets look at them
texts = twitter_archive[twitter_archive.rating_denominator > 100]['text'].values
for i in range(len(texts)):
    print("Text " + str(i+1)+ ":" + texts[i])

> If we look at each one of these tweets, we realize that the score given is for all the dogs that we can observe (who are represented by the denominator * 10)
>
> - Tweets (758467244762497024, 731156023742988288, 684225744407494656, 684222868335505415, 677716515794329600) - Adjust rating

In [ ]:
twitter_archive[(twitter_archive.rating_denominator > 10) & 
                (twitter_archive.rating_denominator < 100)][['tweet_id', 'text', 'rating_numerator', 'rating_denominator']]

In [ ]:
# Both numerator and denominator seem to be pretty high, lets look at the text to see why.
# Something seems odd about text and the numerators, leets look at them
texts = twitter_archive[(twitter_archive.rating_denominator > 10) & 
                (twitter_archive.rating_denominator < 100)]['text'].values
for i in range(len(texts)):
    print("Text " + str(i+1)+ ":" + texts[i])

> **Fact: Dogs who died at the terrorist attack of 9/11 are rated wrong, instead they are granted with one of the highest grades possible: 14/10. This is pretty sad :(**  
>
> - Tweets: (775096608509886464, 740373189193256964, 716439118184652801, 682962037429899265, 722974582966214656) [14/10, 14/10, 11/10, 10/10, 13/10] - Incorrect rating
>
> - Some "ratings" are confused with the date when an account was created. 
> Tweet: (832088576586297345) - Invalid rating
>
> - Same issue as the one that was mentioned for some tweets that involve 2 or more dogs. Tweets: (820690176645140481, 713900603437621249, 710658690886586372, 709198395643068416, 704054845121142784, 697463031882764288, 675853064436391936) - Adjust rating

In [ ]:
# Lets look a the more extreme outliers
twitter_archive[twitter_archive.rating_numerator > 400][['tweet_id', 'text', 'rating_numerator', 'rating_denominator']]

In [ ]:
# Making aside the tweet that was previously analized, the others seem to have normal denominators, lets see why
texts = twitter_archive[twitter_archive.rating_numerator > 400]['text'].values
for i in range(len(texts)):
    print("Text " + str(i+1)+ ":" + texts[i])

**Looking at these tweets we realize something pretty funny about all of them, the one with 1776/10 makes reference to the independence year of USA, the one with 666 makes reference to the "number of the beast", and the ones with 420 are not dogs, but the rapper Snop Dog xD**

> Since they are special cases, we might consider to remove them, or do not take them into account for further analysis.
> Tweets: (855862651834028034, 855860136149123072, 749981277374128128, 670842764863651840)

> **Note: Using NLP for the tweets who present abnormalities is an interesenting option, in order to reduce the time and automatize this activity.**

In [ ]:
# Now we will look at the types of dogs and their names
twitter_archive.name.value_counts()

> This column presents come problems and it will involve just too much time cleaning it...

In [ ]:
twitter_archive.doggo.value_counts()

In [ ]:
twitter_archive.pupper.value_counts()

In [ ]:
twitter_archive.floofer.value_counts()

In [ ]:
twitter_archive.puppo.value_counts()

> Given the values of the previous columns, it will be better to take it as a integer column with 0's and 1's, instead of strings.

In [ ]:
# We will see if have duplicates id in the dataset.
twitter_archive.tweet_id.nunique()

> It seems that for this dataset we do not have duplicates.

### Image predictions

In [ ]:
# Look for dimensions
image_predictions.info()

> - At first sight it seems that there are no missing values, and that each data type is correct.
> - The columns are not informative enough.

In [ ]:
# Now lets verify if there are not duplicated tweets
image_predictions.tweet_id.nunique()

In [ ]:
# We verify the values of the breed columns
print("p1:" + str(image_predictions.p1.nunique()))
print("p2:" + str(image_predictions.p2.nunique()))
print("p3:" + str(image_predictions.p3.nunique()))

In [ ]:
# We verify which values are in the column p1
image_predictions.p1.value_counts()

> There are values that are not breeds and correspond to objects, not dogs. We need to be aware from these value when we select type of breed from p1, p2 or p3.

In [ ]:
# Now lets look at duplicated urls
print(image_predictions[image_predictions.jpg_url.duplicated()].count())
image_predictions[image_predictions.jpg_url.duplicated(keep = False)].sort_values(
    by = 'jpg_url').head()

In [ ]:
# Now we verify those url in the twitter archive and try to guess why they are equal
twitter_archive[twitter_archive.tweet_id.isin([675354435921575936, 752309394570878976, 842892208864923648, 807106840509214720])]

> Here we can notice that the duplicates belong to retweets, so we must do a double check to verify if they were all removed.

### Tweets counts

In [ ]:
# Look for dimensions
tweets_counts.info()

In [ ]:
# Now look for duplicated tweets id
tweets_counts.tweet_id.nunique()

In [ ]:
tweets_counts.describe()

**Now we will look at the values distribution**

In [ ]:
# We will show two plots, one without limits, and other with limit at x=10,000
plt.figure(figsize=[15,10])

plt.subplot(2,2,1)
plt.hist(data = tweets_counts, x = 'retweet_count');
plt.title("Retweet Count Distribution")
plt.xlabel('Retweets')
plt.ylabel('Frequency')

plt.subplot(2,2,2)
plt.hist(data = tweets_counts, x = 'retweet_count', range=(0,10000));
plt.title("Retweet Count Distribution")
plt.xlabel('Retweets')
plt.ylabel('Frequency')

plt.subplot(2,2,3)
plt.hist(data = tweets_counts, x = 'favorite_count');
plt.title("Favorite Count Distribution")
plt.xlabel('Favorite')
plt.ylabel('Frequency')

plt.subplot(2,2,4)
plt.hist(data = tweets_counts, x = 'favorite_count', range=(0,20000));
plt.title("Favorite Count Distribution")
plt.xlabel('Favorite')
plt.ylabel('Frequency');

> From the previous statistics and plots we can see that the distribution is right skewed, with some tweets (outliers) that we should be aware of.

### Observations
### Quality
`twitter_achive table`
- There are rows that belong to retweets, we should get rid of this rows.
- These columns are not needed since they belong to retweets: (retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp, in_reply_to_status_id, in_reply_to_user_id).

- In the name column, the name is not an actual name. There are serveral prepositions that shouldn't be there, like: "the, an, this, etc".
- Incorrect datatype for the column 'timestamp', instead of string it should be datetime.
- Incorrect datatype for the columns *(doggo, floofer, pupper, puppo)* they are string when they only have two possible values, so they should be boolean datatype. 
- The rate assigned to the following tweets is incorrect, they should be corrected. Tweets (835246439529840640, 666287406224695296, 775096608509886464, 740373189193256964, 716439118184652801, 682962037429899265, 722974582966214656) [14/10, 14/10, 11/10, 10/10, 13/10, 13/10, 9/10] - Incorrect rating
- The rate assigned to the following tweets is for all the dogs that appear on it, so the values should be recalculated. Tweets (758467244762497024, 731156023742988288, 684225744407494656, 684222868335505415, 677716515794329600, 820690176645140481, 713900603437621249, 710658690886586372, 709198395643068416, 704054845121142784, 697463031882764288, 675853064436391936) - Adjust rating
- These tweets do not have valid ratings and there is no futher information that we can use, so they should be removed. Tweets (810984652412424192, 855862651834028034, 855860136149123072, 749981277374128128, 670842764863651840 )

`image_predictions table`
- Columns: *(p1, p1_conf, p1_dog, p2, p2_conf, p2_dog, p3, p3_conf, p3_dog)* are not informative enough.
- In the columns *(p1, p2, p3)* all the values should be converted to lower case to avoid misspelling.

### Tidiness
- There are 2075 rows in `image_prediction table`, 2356 in `tweet_archive table`, and 2354 `tweets_count table`. 
- The types of dog corresponds to only one variable which is called the age stage of that dog.
- At the end we only need 1 a table that contains all the data relevant for our analysis (observational unit).
- In `image_prediction table` we only need 1 column for the breed and the result, instead of the 3.

<a id='cleaning'></a>
## Cleaning Data

#### Making a copy of the three datasets

In [ ]:
# First of all we make a copy of the three datasets before we start cleaning
twitter_archivecp = twitter_archive.copy()
image_predictionscp = image_predictions.copy()
tweets_countscp = tweets_counts.copy()

### Missing Data

`twitter_archivecp`

#### Define
- Remove all the rows that belong to retweets.
- Remove columns that are associated with retweets (retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp, in_reply_to_status_id, in_reply_to_user_id).
- Remove all the rows that contain tweets that have no valid ratings: (810984652412424192, 855862651834028034, 855860136149123072, 749981277374128128, 670842764863651840 )

#### Code

In [ ]:
# First we obtain all the tweets that have null values in the parameters that belong to retweets
twitter_archivecp = twitter_archivecp[(twitter_archivecp.retweeted_status_user_id.isnull()) & 
                (twitter_archivecp.retweeted_status_id.isnull()) & 
                (twitter_archivecp.retweeted_status_timestamp.isnull()) & 
                (twitter_archivecp.in_reply_to_status_id.isnull()) & 
                (twitter_archivecp.in_reply_to_user_id.isnull())]

In [ ]:
# Drop all the columns that are associated with retweets
twitter_archivecp.drop(['retweeted_status_id', 'retweeted_status_user_id',
                      'retweeted_status_timestamp', 'in_reply_to_status_id',
                      'in_reply_to_user_id'], inplace=True, axis=1)

In [ ]:
# Finally we remove all the rows that have no valid strings
invalid_tweets = [810984652412424192, 855862651834028034, 855860136149123072, 749981277374128128, 670842764863651840]
indexes = twitter_archivecp.index[twitter_archivecp.tweet_id.isin(invalid_tweets)].tolist()
indexes

In [ ]:
twitter_archivecp.drop(indexes, inplace=True)

#### Test

In [ ]:
twitter_archivecp.info()

In [ ]:
assert twitter_archivecp[twitter_archivecp.tweet_id.isin(invalid_tweets) == True].shape[0]== 0

#### Define

- Replace all the prepositions or words that are not actual names with nulls, format the name.

#### Code

In [ ]:
# All the words that are not names
non_name = ['O', 'a', 'a', 'about', 'above', 'after', 'again', 
             'against', 'all', 'all', 'am', 'an', 'an', 'and', 
             'any', 'are', 'as', 'at', 'at', 'be', 'because', 
             'been', 'before', 'being', 'below', 'between', 
             'both', 'but', 'by', 'by', 'can', 'did', 'do', 
             'does', 'doing', 'don', 'down', 'during', 'each', 
             'few', 'for', 'from', 'further', 'had', 'has', 
             'have', 'having', 'he', 'her', 'here', 'hers', 
             'herself', 'him', 'himself', 'his', 'how', 'i', 
             'if', 'in', 'into', 'is', 'it', 'its', 'itself', 
             'just', 'just', 'life', 'light', 'me', 'more', 
             'most', 'my', 'my', 'myself', 'no', 'nor', 'not', 
             'not', 'now', 'none', 'of', 'off', 'old', 'on', 'once', 
             'only', 'or', 'other', 'our', 'ours', 'ourselves', 
             'out', 'over', 'own', 'quite', 's', 'same', 'she', 
             'should', 'so', 'some', 'space', 'such', 'such', 
             't', 'than', 'that', 'the', 'the', 'their', 'theirs', 
             'them', 'themselves', 'then', 'there', 'these', 'they', 
             'this', 'this', 'those', 'through', 'to', 'too', 'under', 
             'until', 'up', 'very', 'very', 'was', 'we', 'were', 
             'what', 'when', 'where', 'which', 'while', 'who', 
             'whom', 'why', 'will', 'with', 'you', 'your', 'yours', 
             'yourself', 'yourselves']

In [ ]:
# Replace the non name words with nulls and capitalize the first letter
twitter_archivecp.name = twitter_archivecp.name.str.lower()
twitter_archivecp.name.replace(non_name, np.nan, inplace = True)
twitter_archivecp.name = twitter_archivecp.name.str.capitalize()

#### Test

In [ ]:
twitter_archivecp.name.value_counts()

#### Define
- Convert to datetime the datatype of the column 'timestamp'
- Convert to boolean (integer values of 0 and 1) datatype the columns 'doggo', 'floofer', 'pupper', 'puppo'

#### Code

In [ ]:
# Convert to datetime the column timestamp
twitter_archivecp.timestamp = pd.to_datetime(twitter_archivecp.timestamp, infer_datetime_format=True)

In [ ]:
# Convert to boolean the columns 'doggo', 'floofer', 'pupper', 'puppo'
columns = ['doggo', 'floofer', 'pupper', 'puppo']
for column in columns:
    twitter_archivecp[column] = twitter_archivecp[column].replace({column:1, 'None':0})

#### Test

In [ ]:
# We verify that the convertion was sucessful
twitter_archivecp.info()

#### Define
- Correct the ratings for the following tweets: (835246439529840640, 666287406224695296, 775096608509886464, 740373189193256964, 716439118184652801, 682962037429899265, 722974582966214656). Ratings: [14/10, 14/10, 11/10, 10/10, 13/10, 13/10, 9/10].

#### Code

In [ ]:
# First of all we have to verify which tweets were not deleted (retweets) and the we proceed to correct the ratings
verify_tweets = [835246439529840640, 666287406224695296, 775096608509886464, 740373189193256964, 716439118184652801, 682962037429899265, 722974582966214656]

twitter_archivecp[twitter_archivecp.tweet_id.isin(verify_tweets) == True]

In [ ]:
# For the ones who were not deleted we proceed to adjust the ratings
adjust_tweets = [740373189193256964, 722974582966214656, 716439118184652801, 682962037429899265, 666287406224695296]
ratings = [[10,10], [9,10], [13,10], [13,10], [14,10]]
indexes = twitter_archivecp.index[twitter_archivecp.tweet_id.isin(adjust_tweets)].tolist()
indexes

In [ ]:
for i in range(len(indexes)):
    twitter_archivecp.loc[indexes[i], 'rating_numerator'] = ratings[i][0]
    twitter_archivecp.loc[indexes[i], 'rating_denominator'] = ratings[i][1]

#### Test

In [ ]:
# Now we verify that the values had been replaced
twitter_archivecp[twitter_archivecp.tweet_id.isin(verify_tweets) == True]

#### Define
- Recalculate and correct the ratings for the following tweets: (758467244762497024, 731156023742988288, 684225744407494656, 684222868335505415, 677716515794329600, 820690176645140481, 713900603437621249, 710658690886586372, 709198395643068416, 704054845121142784, 697463031882764288, 675853064436391936)

#### Code

In [ ]:
# First of all we have to verify which tweets were not deleted (retweets) and the we proceed to correct the ratings
recalculate_tweets = [758467244762497024, 731156023742988288, 684225744407494656, 684222868335505415, 677716515794329600, 820690176645140481, 713900603437621249, 710658690886586372, 709198395643068416, 704054845121142784, 697463031882764288, 675853064436391936]

twitter_archivecp[twitter_archivecp.tweet_id.isin(recalculate_tweets) == True]

In [ ]:
# Now we extract the indexes and we proceed to recalculate the rating values
indexes = twitter_archivecp.index[twitter_archivecp.tweet_id.isin(recalculate_tweets)].tolist()
indexes

In [ ]:
#We recalculate the values
for index in indexes:
    number_of_dogs = twitter_archivecp.loc[index, 'rating_denominator']/10
    twitter_archivecp.loc[index, 'rating_numerator'] = int(twitter_archivecp.loc[index, 'rating_numerator']/number_of_dogs)
    twitter_archivecp.loc[index, 'rating_denominator'] = 10

#### Test

In [ ]:
twitter_archivecp[twitter_archivecp.tweet_id.isin(recalculate_tweets) == True]

#### Define

`image_predictions table`
- Convert values of columns (p1,p2,p3) to lowercase
- Rename the columns (p1, p1_conf, p1_dog, p2, p2_conf, p2_dog, p3, p3_conf, p3_dog) to (prediction_#, conf_probability_#, predicted_breed_#), where # is the number 1,2 and 3.
- Create two columns (breed, result) were the values with the highest probability will be held.

#### Code

In [ ]:
# We convert the columns 'p1', 'p2' and 'p3' to lowercase
columns = ['p1', 'p2', 'p3']
for column in columns:
    image_predictionscp[column] = image_predictionscp[column].str.lower()

In [ ]:
# Now we rename the columns (p1, p1_conf, p1_dog, p2, p2_conf, p2_dog, p3, p3_conf, p3_dog)
image_predictionscp.rename(columns={"p1": "prediction_1", "p2": "prediction:_2", "p3": "prediction:_3", 
                                    "p1_conf": "conf_probability_1", "p2_conf": "conf_probability_2", 
                                    "p3_conf": "conf_probability_3", "p1_dog": "predicted_breed_1",
                                    "p2_dog": "predicted_breed_2", "p3_dog": "predicted_breed_3",}, inplace=True);

In [ ]:
# We add a new column in which we will save the breed (in case of a dog) or object with the highest probaility
# and another column which states that if it is a dog or not.
# Remove all the rows in which neither of the 3 predictions are dogs
indexes = image_predictionscp.index[((image_predictionscp.predicted_breed_1) | 
                    (image_predictionscp.predicted_breed_2) | 
                    (image_predictionscp.predicted_breed_3)) == False].tolist()


In [ ]:
# Verify that there is no more duplicated urls due to retweets
# Now lets look at duplicated urls
indexes = image_predictionscp[image_predictionscp.jpg_url.duplicated(keep = False)].sort_values(
    by = 'jpg_url').index
len(indexes)

#### Test

In [ ]:
image_predictionscp.info()

In [ ]:
twitter_archivecp[twitter_archivecp.index.isin(indexes)]

> Seems that there are not more retweets :)

<a id='iterating'></a>
## Additional Assessing and Cleaning (Iteration)

### Programmatic Assessment

In [ ]:
# Let's look if there are any rates that are inconsistent
twitter_archivecp.rating_numerator.value_counts()

> There are 3 values that are suspicious, lets dig in!

In [ ]:
twitter_archivecp[twitter_archivecp.rating_numerator > 20]

In [ ]:
texts = twitter_archivecp[twitter_archivecp.rating_numerator > 20]['text'].values

In [ ]:
for i in range(len(texts)):
    print("Text " + str(i+1)+ ":" + texts[i])

> Reading at the text we notice that the problem was that the numerators are the decimal values of the rates.

### Quality
`twitter_archivecp`
- The rate assigned to the following tweets is incorrect, they should be corrected. Tweets: (786709082849828864, 778027034220126208, 680494726643068929) [10/10, 11/10, 11/10]

## Cleaning

#### Define
Correct the ratings for the following tweets: (786709082849828864, 778027034220126208, 680494726643068929). Ratings: [10/10, 11/10, 11/10].

#### Code

In [ ]:
# For the ones who were not deleted we proceed to adjust the ratings
adjust_tweets = [786709082849828864, 778027034220126208, 680494726643068929]
ratings = [[10,10], [11,10], [11,10]]
indexes = twitter_archivecp.index[twitter_archivecp.tweet_id.isin(adjust_tweets)].tolist()
indexes

In [ ]:
for i in range(len(indexes)):
    twitter_archivecp.loc[indexes[i], 'rating_numerator'] = ratings[i][0]
    twitter_archivecp.loc[indexes[i], 'rating_denominator'] = ratings[i][1]

#### Test

In [ ]:
# Now we verify that the values had been replaced
twitter_archivecp[twitter_archivecp.tweet_id.isin(adjust_tweets) == True]

## Tidiness

#### Define
- Group the different types of doggs into only one column named age stage which will contain all the stages of the dog's age.

#### Code

In [ ]:
# Map each stage of the dog that exists: doggo, floofer, puper, puppo
age_stages = ['doggo', 'floofer', 'pupper', 'puppo']
# Function that maps every state of the dogs
def calc_dog_stage(stages):
    for stage in stages:
        if stage in age_stages:
            return stage
        else:
            pass
    return stage

In [ ]:
# Apply the given function
twitter_archivecp['age_stages'] = twitter_archivecp[['doggo', 'floofer',  'pupper', 'puppo']].apply(calc_dog_stage, axis = 1)
# Then we drop the remaining columns
twitter_archivecp.drop(['doggo', 'floofer', 'pupper', 'puppo'], 
    inplace = True, axis = 1)
# Finally we convert that variable as category
twitter_archivecp.age_stages = twitter_archivecp.age_stages.astype('category')

#### Test

In [ ]:
#  We verify that the previous operation was sucessfull
twitter_archivecp.head(0)

#### Define
- In `image_prediction table` create two arrays one with the prediction, and other with the result of the predictions that had the highest probability.

#### Code

In [316]:
# We create a two list, one with the object and other with the result
predictions =[]
results = []

# For loop, were the values with the highest probability are selected and appended
for i in range (image_predictionscp.shape[0]):
    prob1 = image_predictionscp.loc[i]['conf_probability_1']
    prob2 = image_predictionscp.loc[i]['conf_probability_2']
    prob3 = image_predictionscp.loc[i]['conf_probability_3']
    
    if (prob1 > prob2) and (prob1 > prob3):
        predictions.append(image_predictionscp.loc[i]['prediction_1'])
        results.append(image_predictionscp.loc[i]['predicted_breed_1'])
    elif (prob2 > prob1) and (prob2 > prob3):
        predictions.append(image_predictionscp.loc[i]['prediction_2'])
        results.append(image_predictionscp.loc[i]['predicted_breed_2'])
    else:
        predictions.append(image_predictionscp.loc[i]['prediction_3'])
        results.append(image_predictionscp.loc[i]['predicted_breed_3'])

In [317]:
# Creation of the columns: 'predictions' and 'results'
image_predictionscp['predictions'] = predictions
image_predictionscp['results'] = results

#### Test

In [318]:
# We visualize the creation of the columns
image_predictionscp.head()

,tweet_id,jpg_url,img_num,prediction_1,conf_probability_1,predicted_breed_1,prediction:_2,conf_probability_2,predicted_breed_2,prediction:_3,conf_probability_3,predicted_breed_3,predictions,results
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,welsh_springer_spaniel,0.465074,True,collie,0.156665,True,shetland_sheepdog,0.061428,True,welsh_springer_spaniel,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,rhodesian_ridgeback,0.072010,True,redbone,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,german_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True,german_shepherd,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True,rhodesian_ridgeback,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,rottweiler,0.243682,True,doberman,0.154629,True,miniature_pinscher,True


#### Define

- Combine the 3 tables together into a single table named: `twitter_archive_master.csv`.
- Drop the columns that will be not part of the analysis.

#### Code

In [320]:
twitter_archive_master = twitter_archivecp.merge(tweets_countscp, on = 'tweet_id', how = 'left')
twitter_archive_master = twitter_archive_master.merge(image_predictionscp[['tweet_id','jpg_url', 'img_num', 'predictions', 'results']], 
  on = 'tweet_id')

In [ ]:
twitter_archive_master.drop(['source', 'text', 'expanded_urls'], 
  inplace = True, axis = 1)

#### Test

In [323]:
twitter_archive_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1968 entries, 0 to 1967
Data columns (total 12 columns):
tweet_id              1968 non-null int64
timestamp             1968 non-null datetime64[ns]
rating_numerator      1968 non-null int64
rating_denominator    1968 non-null int64
name                  1358 non-null object
age_stages            1968 non-null category
retweet_count         1968 non-null int64
favorite_count        1968 non-null int64
jpg_url               1968 non-null object
img_num               1968 non-null int64
predictions           1968 non-null object
results               1968 non-null bool
dtypes: bool(1), category(1), datetime64[ns](1), int64(6), object(3)
memory usage: 173.1+ KB


In [324]:
twitter_archive_master.head()

,tweet_id,timestamp,rating_numerator,rating_denominator,name,age_stages,retweet_count,favorite_count,jpg_url,img_num,predictions,results
0,892420643555336193,2017-08-01 16:23:56,13,10,Phineas,0,8853,39467,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,1,orange,False
1,892177421306343426,2017-08-01 00:17:27,13,10,Tilly,0,6514,33819,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,chihuahua,True
2,891815181378084864,2017-07-31 00:18:03,12,10,Archie,0,4328,25461,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,chihuahua,True
3,891689557279858688,2017-07-30 15:58:51,13,10,Darla,0,8964,42908,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,False
4,891327558926688256,2017-07-29 16:00:24,12,10,Franklin,0,9774,41048,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,True


## Store the data

Finally we have our dataset cleaned and ready to be stored, we will use SQL Lite as DBMS to create a database and store `twitter_archive table`.

In [344]:
# First of all, we create a backup for the dataframes cleaned
folder_name = 'bck'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Twitter Archive Clenaned Dataframe
twitter_archivecp.to_csv('bck/twitter_archivecp.csv', encoding='utf-8', index=False)
# Image Predictions Clenaned Dataframe
image_predictionscp.to_csv('bck/image_predictionscp.csv', encoding='utf-8', index=False)
# Tweets Counts Cleaned Dataframe
tweets_countscp.to_csv('bck/tweets_countscp.csv', encoding='utf-8', index=False)
# Twitter Archive Master Clenaned Dataframe
twitter_archive_master.to_csv('bck/twitter_archive_master.csv', encoding='utf-8', index=False)

#### Now we save the master dataframe in disk, using a database

In [347]:
# Connect to the sqlite database
database = 'wrangle_act.db'
conn = sqlite3.connect(database)
twitter_archive_master.to_sql('bck/twitter_archive_master', conn, if_exists = 'replace', index = False)

#### Finally we test the database, by making a select statement

In [348]:
# Make sure they all read back from the database
twitter_archive_master = pd.read_sql('SELECT * FROM twitter_archive_master', conn) 

DatabaseError: Execution failed on sql 'SELECT * FROM twitter_archive_master': no such table: twitter_archive_master